In [1]:
import os
import logging
import json
import pickle
import torch
import numpy as np
import pandas as pd
import xarray as xr
import seqdata as sd
import seqpro as sp
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed')).History will not be written to the database.


/cellar/users/aklie/opt/miniconda3/envs/eugene_tools/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from bpnetlite.chrombpnet import BPNet

In [3]:
from eugene.utils import (
    merge_parameters,
    check_for_gpu,
)

In [4]:
fold = "fold_0"
path_seqdata = "/cellar/users/aklie/projects/ML4GLand/tutorials/bulk_atac_basepair/eugene/prep_dataset/K562_ATAC-seq.minimal.seqdata"

In [5]:
seq_var = "seq"
cov_var = "cov" 
seq_length = 2114
target_length = 1000
max_jitter = 0
cov_var = "cov"
max_counts = 103
min_counts = 0
outlier_threshold = 0.9999

In [6]:
n_filters = 128
n_layers = 4
n_outputs = 1
n_control_tracks = 0
alpha = None

In [35]:
learning_rate = 0.001
batch_size = 64
max_epochs = 2
validation_iter = 100
rc_augment = True

In [8]:
sdata = sd.open_zarr(path_seqdata)

In [9]:
trimming = (seq_length - target_length) // 2
seqs_start = (sdata.dims["_length"] // 2) - (seq_length // 2)
counts_start = (sdata.dims["_length"] // 2) - (target_length // 2)
seqs_start, counts_start

(512, 1069)

In [ ]:
training_idx = np.where(sdata[fold].isin(["train", "valid"]))[0]
test_idx = np.where(sdata[fold] == "test")[0]

In [11]:
training_data = sdata.isel(_sequence=training_idx)
test_data = sdata.isel(_sequence=test_idx)
print(training_data.dims["_sequence"], test_data.dims["_sequence"])

432140 107270


In [12]:
training_cov = training_data[cov_var].values
training_counts = training_cov[..., counts_start:counts_start + target_length].sum(axis=(1,2))

In [13]:
cnts_msk = (training_counts <= max_counts) & (training_counts >= min_counts)
filt_cnts = training_counts[cnts_msk]
if outlier_threshold is not None:
    upper_thresh = np.quantile(filt_cnts, outlier_threshold)
    lower_thresh = np.quantile(filt_cnts, 1 - outlier_threshold)
    outlier_msk = (training_counts <= upper_thresh) & (training_counts >= lower_thresh)

In [14]:
print(max_counts, min_counts, outlier_threshold)

103 0 0.9999


In [15]:
print(upper_thresh, lower_thresh)

103.0 0.0


In [16]:
cnts_msk.sum(), outlier_msk.sum()

(228896, 228896)

In [17]:
training_data = training_data.isel(_sequence=outlier_msk)
training_data

<xarray.Dataset> Size: 4GB
Dimensions:     (_sequence: 228896, cov_sample: 1, _length: 3138)
Coordinates:
  * _sequence   (_sequence) <U15 14MB 'negative_14132' ... 'negative_269691'
  * cov_sample  (cov_sample) object 8B 'SC.delta'
Dimensions without coordinates: _length
Data variables: (12/18)
    blockCount  (_sequence) float64 2MB dask.array<chunksize=(19585,), meta=np.ndarray>
    chrom       (_sequence) object 2MB dask.array<chunksize=(53285,), meta=np.ndarray>
    chromEnd    (_sequence) int64 2MB dask.array<chunksize=(19585,), meta=np.ndarray>
    chromStart  (_sequence) int64 2MB dask.array<chunksize=(19585,), meta=np.ndarray>
    cov         (_sequence, cov_sample, _length) float32 3GB dask.array<chunksize=(150, 1, 3138), meta=np.ndarray>
    fold_0      (_sequence) <U5 5MB dask.array<chunksize=(19576,), meta=np.ndarray>
    ...          ...
    score       (_sequence) float64 2MB dask.array<chunksize=(19585,), meta=np.ndarray>
    seq         (_sequence, _length) |S1 718MB dask.array<chunksize=(150, 3138), meta=np.ndarray>
    strand      (_sequence) object 2MB dask.array<chunksize=(53285,), meta=np.ndarray>
    thickEnd    (_sequence) float64 2MB dask.array<chunksize=(19585,), meta=np.ndarray>
    thickStart  (_sequence) float64 2MB dask.array<chunksize=(19585,), meta=np.ndarray>
    type        (_sequence) <U8 7MB dask.array<chunksize=(2719,), meta=np.ndarray>
Attributes:
    length_dim:    _length
    max_jitter:    512
    sequence_dim:  _sequence

In [18]:
if alpha is None:
    alpha = np.median(training_counts[outlier_msk]) / 10
    print(alpha)

5.9


In [19]:
train_idx = np.where(training_data[fold] == "train")[0]
train_data = training_data.isel(_sequence=train_idx).load()

In [20]:
arch = BPNet(
    n_filters=n_filters,
    n_layers=n_layers,
    n_outputs=n_outputs,
    n_control_tracks=n_control_tracks,
    trimming=trimming,
    alpha=alpha,
    name="test",
    verbose=True,
)

In [21]:
rng = np.random.default_rng(1234)

In [ ]:
def transform(batch):
    if max_jitter > 0:
        batch['seq'], batch['cov'] = sp.jitter(batch['seq'], batch["cov"], max_jitter=max_jitter, length_axis=-1, jitter_axes=0)  # jitter
        batch['cov'] = batch['cov'][..., trimming:-trimming]  # trim
    else:
        batch['seq'] = batch['seq'][..., seqs_start:seqs_start + seq_length]
        batch['cov'] = batch['cov'][..., counts_start:counts_start + target_length]
    batch['seq'] = sp.DNA.ohe(batch['seq']).transpose(0, 2, 1)  # one hot encode
    if rng.choice(2) == 1:
        batch['seq'] = sp.reverse_complement(batch['seq'], alphabet=sp.DNA, length_axis=-1, ohe_axis=1).copy()
        batch['cov'] = np.flip(batch['cov'], axis=-1).copy()
    return batch

In [27]:
vars = [seq_var, cov_var]

In [28]:
train_dl = sd.get_torch_dataloader(
    train_data,
    sample_dims=['_sequence'],
    variables=vars,
    prefetch_factor=None,
    transform=transform,
    batch_size=batch_size,
    shuffle=True,
    return_tuples=True
)

In [29]:
batch = next(iter(train_dl))
print(f"Batch looks like: {batch[0].shape}, {batch[1].shape}")

Batch looks like: torch.Size([64, 4, 2114]), torch.Size([64, 1, 1000])


In [31]:
valid_idx = np.where(training_data[fold] == "valid")[0]
valid_data = training_data.isel(_sequence=valid_idx).load()
X_valid = torch.tensor(sp.ohe(valid_data["seq"].values[:, seqs_start:seqs_start + seq_length], alphabet=sp.DNA).transpose(0, 2, 1), dtype=torch.float32)
y_valid = torch.tensor(valid_data["cov"].values[..., counts_start:counts_start + target_length], dtype=torch.float32)
X_valid.shape, y_valid.shape

(torch.Size([21415, 4, 2114]), torch.Size([21415, 1, 1000]))

In [32]:
arch.cuda()

BPNet(
  (iconv): Conv1d(4, 128, kernel_size=(21,), stride=(1,), padding=(10,))
  (irelu): ReLU()
  (rconvs): ModuleList(
    (0): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
    (1): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,))
    (2): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(8,), dilation=(8,))
    (3): Conv1d(128, 128, kernel_size=(3,), stride=(1,), padding=(16,), dilation=(16,))
  )
  (rrelus): ModuleList(
    (0-3): 4 x ReLU()
  )
  (fconv): Conv1d(128, 1, kernel_size=(75,), stride=(1,), padding=(37,))
  (linear): Linear(in_features=128, out_features=1, bias=True)
)

In [33]:
optimizer = torch.optim.Adam(arch.parameters(), lr=learning_rate)

In [34]:
arch.fit(
    train_dl,
    optimizer,
    X_valid=X_valid,
    y_valid=y_valid,
    X_ctl_valid=None,
    max_epochs=max_epochs,
    validation_iter=validation_iter,
)

Epoch	Iteration	Training Time	Validation Time	Training MNLL	Training Count MSE	Validation MNLL	Validation Profile Pearson	Validation Count Pearson	Validation Count MSE	Saved?
0	0	1.5323	2.4215	224.2034	16.4881	220.05	0.124981284	0.05528001	9.2166	True
0	100	4.5863	2.2785	170.0277	0.7665	167.0724	0.36987892	0.11394153	0.9929	True
0	200	4.4183	2.1715	145.1361	2.1809	164.3018	0.38562426	0.16811876	1.1836	True
0	300	4.3125	2.1547	151.0286	0.6813	163.3	0.38995412	0.22862452	1.2489	True
0	400	4.3657	2.1578	143.3384	1.7597	162.7032	0.39421085	0.2864998	0.9742	True
0	500	4.3638	2.1732	161.2723	0.9908	162.6994	0.39610305	0.32443577	1.2558	False
0	600	4.3742	2.1584	151.7823	1.2467	162.2676	0.39716685	0.35142353	0.9591	True
0	700	4.3069	2.1605	148.5048	1.8502	162.1134	0.39748743	0.40741408	0.96	True
0	800	4.3117	2.1785	150.3658	1.0482	162.1929	0.39764482	0.41938117	0.9809	False
0	900	4.3193	2.1804	158.6173	1.118	161.9579	0.39891377	0.4633377	0.9666	True
0	1000	4.3766	2.1651	145.7594	1.0284	161.73